In [46]:
from io import StringIO
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

import pandas as pd
import numpy as np

# Dealing with missing data

In [2]:
csv_data = \
    '''A,B,C,D
    1.0,2.0,3.0,4.0
    5.0,6.0,,8.0
    10.0,11.0,12.0,'''

df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [5]:
# isnull() returns a dataset with boolean values, where True indicates a missing value
# then we sum the number of missing values for each column
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

In [9]:
# Returns numpy array from a dataset, most of scikit-learn functions expect numpy arrays
print(df.values)

[[ 1.  2.  3.  4.]
 [ 5.  6. nan  8.]
 [10. 11. 12. nan]]


In [15]:
# Remove missing values

# remove rows/samples with missing values
print(df.dropna(axis=0))

# remove columns/features with missing values
print(df.dropna(axis=1))

# only removes columns if all rows for that column are NaN
print(df.dropna(how='all'))

# drop rows with fewer than 4 real values
print(df.dropna(thresh=4))

# drop rows where NaN appears in specific columns
print(df.dropna(subset=['C']))

     A    B    C    D
0  1.0  2.0  3.0  4.0
      A     B
0   1.0   2.0
1   5.0   6.0
2  10.0  11.0
      A     B     C    D
0   1.0   2.0   3.0  4.0
1   5.0   6.0   NaN  8.0
2  10.0  11.0  12.0  NaN
     A    B    C    D
0  1.0  2.0  3.0  4.0
      A     B     C    D
0   1.0   2.0   3.0  4.0
2  10.0  11.0  12.0  NaN


In [23]:
# Imputing missing values

# A common method for imputing value is to use interpolation, like mean imputation (which inputs a value that is the mean of the column)
# Other strategies are median and most_frequent
# Most frequent is useful for categorical data, for example: missing colors
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df.values)  # Notice the conversion of df to numpy array using df.values
imputed_data = imr.transform(df.values)
print(imputed_data)

# A more convenient method using pandas
print(df.fillna(df.mean()))

[[ 1.   2.   3.   4. ]
 [ 5.   6.   7.5  8. ]
 [10.  11.  12.   6. ]]
      A     B     C    D
0   1.0   2.0   3.0  4.0
1   5.0   6.0   7.5  8.0
2  10.0  11.0  12.0  6.0


# Handling categorical data

In [31]:
df = pd.DataFrame([
    ['green', 'M', 10.1, 'class2'],
    ['red', 'L', 13.5, 'class1'],
    ['blue', 'XL', 15.3, 'class2']])
# Color is a nominal feature (not order between the values)
# Size is an ordinal feature (there is an order between the values)
# Price is a numerical feature
df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


In [25]:
# To handle nominal features, we need to convert them to some numerical representation with ordering
size_mapping = {'XL': 3, 'L': 2, 'M': 1}  # because of the order we have to hard-code the mapping
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [26]:
# To reverse the mapping we can do
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

In [40]:
# For class labels we also need to convert, but classes are not ordinal, and so we don't need to worry about the order
class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
df['classlabel'] = df['classlabel'].map(class_mapping)
print("Label mapping:\n", df)

# and to reverse the mapping
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
print("Revert mapping:\n", df)

# Convenient function from scikit-learn
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)  # A shortcut from calling fit and transform separately
print("SciKit label mapping:\n", y)
print("SciKit revert mapping:\n", class_le.inverse_transform(y))

Label mapping:
    color size  price  classlabel
0  green    M   10.1           1
1    red    L   13.5           0
2   blue   XL   15.3           1
Revert mapping:
    color size  price classlabel
0  green    M   10.1     class2
1    red    L   13.5     class1
2   blue   XL   15.3     class2
SciKit label mapping:
 [1 0 1]
SciKit revert mapping:
 ['class2' 'class1' 'class2']


In [51]:
# One-hot encoding
X = df[['color', 'size', 'price']].values
color_le = LabelEncoder()
print("Original\n", X)

X[:, 0] = color_le.fit_transform(X[:, 0])
print("Encoded\n", X)

# Using the encoded data for colors will confuse the algorithm by thinking 
# these values are ordinal (have order) when in reality they don't
# A common technique to handle this is to use one-hot encoding
color_ohe = OneHotEncoder() # To handle multi-collinearity use: categories='auto', drop='first'
print("One hot for colors column only\n", color_ohe.fit_transform(X[:, 0].reshape(-1, 1)).toarray())

# Modifying with one-hot a dataset
X = df[['color', 'size', 'price']].values
c_transf = ColumnTransformer([
    ('one_hot', OneHotEncoder(), [0]),  # Modify only the first column
    ('nothing', 'passthrough', [1, 2])  # Do nothing to the other columns
])
print("One hot in dataset\n", c_transf.fit_transform(X))

# Even more convenient is pd.get_dummies, which automatically applies for all string columns
print("Pandas:\n", pd.get_dummies(df[['price', 'color', 'size']]))

# To avoid multi-collinearity which is a problem that raises during matrix multiplicate with many columns which are
# dependent on each other we can remove the first column of the one-hot encoding
print("Pandas without multi-collinearity:\n", pd.get_dummies(df[['price', 'color', 'size']], drop_first=True))

Original
 [['green' 'M' 10.1]
 ['red' 'L' 13.5]
 ['blue' 'XL' 15.3]]
Encoded
 [[1 'M' 10.1]
 [2 'L' 13.5]
 [0 'XL' 15.3]]
One hot for colors column only
 [[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
One hot in dataset
 [[0.0 1.0 0.0 'M' 10.1]
 [0.0 0.0 1.0 'L' 13.5]
 [1.0 0.0 0.0 'XL' 15.3]]
Pandas:
    price  color_blue  color_green  color_red  size_L  size_M  size_XL
0   10.1       False         True      False   False    True    False
1   13.5       False        False       True    True   False    False
2   15.3        True        False      False   False   False     True
Pandas without multi-collinearity:
    price  color_green  color_red  size_M  size_XL
0   10.1         True      False    True    False
1   13.5        False       True   False    False
2   15.3        False      False   False     True
